In [1]:
import pandas as pd 
import numpy as np
import pandas as pd 
import numpy as np
import torch
import csv
import numpy as np
import pandas as pd
import re
import nltk
import spacy
import string
import re
from bs4 import BeautifulSoup
import contractions
import re
import unicodedata
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import Vocab
from transformers import BertTokenizer
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm  # For a nice progress bar!
from torch.utils.data import Dataset, DataLoader

from os import listdir
from os.path import isfile, join

In [2]:
df = pd.read_csv('sample_0.02_33.csv',dtype={'OwnerUserId': 'float64'})

/opt/homebrew/Caskroom/miniforge/base/envs/pytorch_env/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df['closed'] = df['ClosedDate'].notnull().astype(int)

In [4]:

df['closed'].value_counts()

0    1059154
1     992131
Name: closed, dtype: int64

In [5]:
df['ClosedDate'] = pd.to_datetime( df['ClosedDate'])
df['CreationDate'] = pd.to_datetime( df['CreationDate'])

In [6]:
# Filtering datasets

startDate = '01-07-2013'
df = df[df['CreationDate'] >= startDate ] 


In [7]:
df['closed'].value_counts()

1    832834
0    828045
Name: closed, dtype: int64

In [11]:
closed_reason_df = pd.read_csv('/Users/tushar/iisc/sem3/TSE/project/ClosedPosts-Type/closed_reasons.csv')



In [16]:
closed_reason_df.head(10)

,RowID,id,comment
0,1,17396037,101
1,2,17396048,104
2,3,17396074,103
3,4,17396115,102
4,5,17396139,102
5,6,17396346,101
6,7,17396354,102
7,8,17396390,102
8,9,17396418,103
9,10,17396469,105


In [23]:
# left joining dataset with closed question dataset to obtain reason for closing 
merged_df = pd.merge(df,closed_reason_df,left_on = 'Id',right_on = 'id', how = 'left')

In [54]:
# Removing posts which were closed in past but now open and by-mistakenly added in closed_reason_df

merged_df = (merged_df[~((merged_df['comment'].notnull()) & (merged_df['closed'] == 0 ))])

In [62]:
# Removing duplicate closed questions 
merged_df = merged_df[~(merged_df['comment'] == 101.0)]

In [70]:
# Replaing values with proper class labels 
merged_df['comment'] = merged_df['comment'].replace(np.nan,0)
merged_df['comment'] = merged_df['comment'].replace(102.0,1)
merged_df['comment'] = merged_df['comment'].replace(103.0,2)
merged_df['comment'] = merged_df['comment'].replace(104.0,3)
merged_df['comment'] = merged_df['comment'].replace(105.0,4)

In [72]:
# converting to ints
merged_df['comment'] = merged_df['comment'].astype(int)


In [78]:
# Generating final dataset
merged_df_no_nans = merged_df.replace(np.nan,"") 

In [79]:
# Keeping only Questions i.e removing Answers, wiki, comments etc
merged_df_no_nans = merged_df_no_nans[ merged_df_no_nans['PostTypeId'] == 1 ]

In [ ]:
merged_df_no_nans['closed'].value_counts()

0    335824
1    330752
Name: closed, dtype: int64

In [85]:
final_df = merged_df_no_nans

In [96]:
final_df[ (final_df['closed'] != 0) & (final_df['comment'] == 0 ) ]

,Unnamed: 0,Id,PostTypeId,Title,Tags,Body,CreationDate,ClosedDate,closed,RowID,id,comment
828059,2580,14188150,1,Installing Pygame on Mac OS X using Homebrew,<python><installation><pygame>,"<p>I'm following <a href=""http://jalada.co.uk/...",2013-01-07 00:02:09.623,2020-12-22 08:11:03.147,1,,,0
828060,2627,14188212,1,"delay when redirecting with php header(""Refresh"")",<php><html>,<blockquote>&#xA; <p><strong>Possible Duplica...,2013-01-07 00:11:49.603,2013-01-07 01:04:13.420,1,,,0
828061,2670,14188269,1,C language & | ~ << >>,<c>,<p>I just learning C. Could someone explain th...,2013-01-07 00:21:43.097,2013-01-07 07:13:47.003,1,,,0
828062,2695,14188296,1,Python programming assistance,<python><python-3.x><computer-science>,<blockquote>&#xA; <p><strong>Possible Duplica...,2013-01-07 00:26:10.467,2013-01-07 00:31:44.620,1,,,0
828063,2724,14188337,1,PHP- Short code to clear screen if certain par...,<php><html>,<p>Lets say I have a simple php index website ...,2013-01-07 00:32:17.647,2013-01-07 00:58:16.090,1,,,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1666018,1312,69060726,1,how to write this code without using any funct...,<c++>,<p>/*i have a project so far i thin im doing g...,2021-09-05 05:21:31.253,2021-09-05 05:40:39.177,1,,,0
1666020,1357,69060772,1,How can extract a hidden product link of a sto...,<web><web-scraping><bots>,<p><strong>Store page example:</strong> <a hre...,2021-09-05 05:30:57.353,2021-09-05 06:20:32.367,1,,,0
1666022,1443,69060859,1,"What does "">location.href"" Script does here?",<javascript><html>,<p>So I have this Spam/adware/phishing link an...,2021-09-05 05:48:08.093,2021-09-05 05:51:09.543,1,,,0
1666023,1471,69060891,1,What is the least case Binary function?,<algorithm><binary-search>,<p>I have learnt about binary search.&#xA;Let'...,2021-09-05 05:57:52.653,2021-09-05 06:20:15.327,1,,,0


In [89]:
final_df['comment'].value_counts()

0    401413
1    121387
2     63013
3     59072
4     21691
Name: comment, dtype: int64

In [98]:
'''
checking how many closed questions were generated during march 2019 till now. 
because author mentioned that 207,040 questions from June 2013 to March 2019
'''

len(final_df[final_df['closed'] == 1]) - len(final_df[(final_df['closed'] == 1 ) & (final_df['CreationDate'] >= '05-03-2019') ])

261615

# We have 330752 closed questions and 335824 open questions to train on 


In [99]:
final_df.to_csv('final_df.csv')

In [82]:
merged_df_no_nans['closed'].value_counts()

0    335824
1    330752
Name: closed, dtype: int64

In [49]:
# df["Title"] = df["Title"].apply(strip_html_tags).apply(remove_accented_chars).apply(contractions.fix).apply(remove_special_characters).apply(ps.stem).apply(lambda x: remove_stopwords(x))

/opt/homebrew/Caskroom/miniforge/base/envs/pytorch_env/lib/python3.9/site-packages/bs4/__init__.py:431: MarkupResemblesLocatorWarning: "http://localhost:8080/webpack-dev-server/" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(


In [55]:
# df["Body"] = df["Body"].apply(strip_html_tags).apply(remove_accented_chars).apply(contractions.fix).apply(remove_special_characters).apply(ps.stem).apply(lambda x: remove_stopwords(x))
# df["Tags"] = df["Tags"].apply(strip_html_tags).apply(remove_accented_chars).apply(contractions.fix).apply(remove_special_characters).apply(ps.stem).apply(lambda x: remove_stopwords(x))

/opt/homebrew/Caskroom/miniforge/base/envs/pytorch_env/lib/python3.9/site-packages/bs4/__init__.py:431: MarkupResemblesLocatorWarning: "https://www.nuget.org/packages/FakeDbSet" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(


### Trying Vainilla BERT Tokenizer and RNN 

In [7]:
# Encode the sentence
tz = BertTokenizer.from_pretrained("bert-base-cased")
def tokenize(df,max_length):
    input_ids = []
    for sent in df:
        encoded = tz.encode_plus(
            text=sent,  # the sentence to be encoded
            add_special_tokens=True,  # Add [CLS] and [SEP]
            max_length = max_length,  # maximum length of a sentence
            pad_to_max_length=True,  # Add [PAD]s
            return_attention_mask = True,  # Generate the attention mask
            truncation=True
        )
        # Get the input IDs and attention mask in tensor format
        input_ids.append(encoded['input_ids'])
        
        
    return torch.tensor(input_ids)
        
    

In [91]:
df['Tags'].str.len().describe()


count    884151.000000
mean         29.624135
std          15.561090
min           3.000000
25%          18.000000
50%          28.000000
75%          39.000000
max         126.000000
Name: Tags, dtype: float64

In [8]:
title_max_length = 150
body_max_length = 1500 
tags_max_length = 150 
df.replace(np.nan, '', inplace=True)
title_tensor = tokenize(df['Title'],title_max_length)
body_tensor = tokenize(df['Body'],body_max_length)
tags_tensor = tokenize(df['Tags'],tags_max_length)

/opt/homebrew/Caskroom/miniforge/base/envs/pytorch_env/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2198: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [11]:
torch.save(title_tensor, 'title_tensor.pt')
torch.save(body_tensor, 'body_tensor.pt')
torch.save(tags_tensor, 'tags_tensor.pt')

In [24]:
torch.save(tz.vocab,'vocab.v')

In [39]:
len(df['closed'])

2111792

In [50]:
class CustomTextDataset(Dataset):
    def __init__(self, txt, labels):
        self.labels = labels
        self.text = txt
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        label = self.labels[idx]
        text = self.text[idx]
#         sample = {"Text": text, "Class": label}
        return (text,label)

# define data set object
dataset = CustomTextDataset(title_tensor,df['closed'].to_numpy())
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

In [77]:

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
input_size = 256
hidden_size = 256
num_layers = 2
num_classes = 2
sequence_length = title_tensor.size(1)
learning_rate = 0.005
batch_size = 2048
num_epochs = 3

# Recurrent neural network (many-to-one)
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes,num_embeddings,embedding_size = 256):
        super(RNN, self).__init__()
        self.emb = nn.Embedding(num_embeddings,embedding_size)
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(embedding_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size * sequence_length, num_classes)

    def forward(self, x):
        # Set initial hidden and cell states
        
        x = self.emb(x)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        # Forward propagate LSTM
        out, _ = self.rnn(x, h0)
        out = out.reshape(out.shape[0], -1)

        # Decode the hidden state of the last time step
        out = self.fc(out)
        return out



In [78]:
# Initialize network (try out just using simple RNN, or GRU, and then compare with LSTM)
model = RNN(input_size, hidden_size, num_layers, num_classes,len(tz.vocab)).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

# Train Network
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
        # Get data to cuda if possible
        data = data.to(device=device).squeeze(1)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent update step/adam step
        optimizer.step()

        

# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0

    # Set model to eval
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device).squeeze(1)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    # Toggle model back to train
    model.train()
    return num_correct / num_samples


print(f"Accuracy on training set: {check_accuracy(train_loader, model)*100:2f}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f}")

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 825/825 [47:36<00:00,  3.46s/it]


Accuracy on training set: 98.035255
Accuracy on test set: 98.00


In [79]:
torch.save(model)

TypeError: save() missing 1 required positional argument: 'f'

In [81]:
torch.save(model,"RNN_title_0.model")